In [1]:
import quimb as qu
import quimb.tensor as qtn
import numpy as np

In [2]:
data = qu.bell_state('psi-').reshape(2, 2)
inds = 'k0', 'k1'
tags = 'KET'

ket = qtn.Tensor(data, inds, tags)
ket

Tensor(shape=(2, 2), inds=('k0', 'k1'), tags={'KET'})

In [3]:
ket.H

Tensor(shape=(2, 2), inds=('k0', 'k1'), tags={'KET'})

In [4]:
ket.data

[[ 0.      +0.j  0.707107+0.j]
 [-0.707107+0.j  0.      +0.j]]

In [5]:
X = qtn.Tensor(qu.pauli('X'), inds=('k0', 'b0'), tags=['PAULI', 'X', '0'])
Y = qtn.Tensor(qu.pauli('Y'), inds=('k1', 'b1'), tags=['PAULI', 'Y', '1'])

In [6]:
Y.data

[[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]

In [7]:
Y.H.data

[[ 0.-0.j -0.+1.j]
 [ 0.-1.j  0.-0.j]]

In [8]:
Y.conj().data

[[ 0.-0.j -0.+1.j]
 [ 0.-1.j  0.-0.j]]

In [9]:
bra = qtn.Tensor(qu.rand_ket(4).reshape(2, 2), inds=('b0', 'b1'), tags={'BRA'})

In [10]:
TN = ket.H & X & Y & bra
print(TN)

TensorNetwork([
    Tensor(shape=(2, 2), inds=('k0', 'k1'), tags={'KET'}),
    Tensor(shape=(2, 2), inds=('k0', 'b0'), tags={'PAULI', 'X', '0'}),
    Tensor(shape=(2, 2), inds=('k1', 'b1'), tags={'Y', 'PAULI', '1'}),
    Tensor(shape=(2, 2), inds=('b0', 'b1'), tags={'BRA'}),
])


In [11]:
L = 44
zeros = '0' * ((L - 2) // 3)
binary = zeros + '1' + zeros + '1' + zeros
print('psi0:', f"|{binary}>")

psi0: |00000000000000100000000000000100000000000000>


In [12]:
psi0 = qtn.MPS_computational_state(binary)
psi0.show()  # prints ascii representation of state

     1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1    
    >->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->->- ...
    | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |     
                                 ...                                  
     1 1 1 1 1 1 1 1 
... >->->->->->->->-<
    | | | | | | | | |


In [13]:
H = qtn.NNI_ham_heis(L)

In [14]:
tebd = qtn.TEBD(psi0, H)

# Since entanglement will not grow too much, we can set quite
#     a small cutoff for splitting after each gate application
tebd.split_opts['cutoff'] = 1e-12

In [15]:
# times we are interested in
ts = np.linspace(0, 80, 101)

mz_t_j = []  # z-magnetization
be_t_b = []  # block entropy
sg_t_b = []  # schmidt gap

# range of bonds, and sites
js = np.arange(0, L)
bs = np.arange(1, L)

In [16]:
# generate the state at each time in ts
#     and target error 1e-3 for whole evolution
for psit in tebd.at_times(ts, tol=1e-3):
    mz_j = []
    be_b = []
    sg_b = []

    # there is one more site than bond, so start with mag
    #     this also sets the orthog center to 0
    mz_j += [psit.magnetization(0)]

    for j in range(1, L):
        # after which we only need to move it from previous site
        mz_j += [psit.magnetization(j, cur_orthog=j - 1)]
        be_b += [psit.entropy(j, cur_orthog=j)]
        sg_b += [psit.schmidt_gap(j, cur_orthog=j)]

    mz_t_j += [mz_j]
    be_t_b += [be_b]
    sg_t_b += [sg_b]

t=80, max-bond=15: 100%|##########| 101/101 [03:25<00:00,  2.03s/it]


In [17]:
tebd.pt.show()

     2 4 5 6 7 8 9 10 11 12 13 14 14 14 15 15 15 15 15 15 15 15 15 15 15 1    
    >->->->->->->->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->-->- ...
    | | | | | | | |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |     
                                 ...                                  
    5 14 14 14 14 14 13 13 12 11 10 9 8 7 6 5 4 2 
... ->-->-->-->-->-->-->-->-->-->-->->->->->->->-o
     |  |  |  |  |  |  |  |  |  |  | | | | | | | |


In [18]:
tebd.err  #  should be < tol=1e-3

0.0009938931328611381

In [19]:
H = qtn.MPO_ham_heis(L)
print("Initial energy:", qtn.expec_TN_1D(psi0.H, H, psi0))
print("Final energy:", qtn.expec_TN_1D(tebd.pt.H , H, tebd.pt))

Initial energy: 8.75
Final energy: (8.74999946929479+1.8323012330676444e-17j)


In [20]:
tebd.pt.H @ tebd.pt

(0.9999999998801925+1.1408058114303859e-16j)